# A First Encounter with Computational Quantum Chemistry

## Introduction

In recent years, the field of computational chemistry has had massive success. However, as progress in computational chemistry accelerates, the nascent field's demand for computational resources has increased exponentially. The emerging field of computational quantum chemistry was established to help bridge the gap between the demand and availability of computational power for computational chemistry-related simulations, aiming to exploit quantum entanglement and other quantum phenomena through quantum computing to simulate the quantum mechanics of chemical systems more accurately and rapidly than what is currently possible with classical simulation.

A promising application of quantum computing to open problems in computational chemistry lies in finding the possible electronic configurations for a given molecule. You may remember from introductory chemistry that the nuclei of atoms are usually surrounded by regions of space, known as orbitals, which contain the atom's electrons. According to Molecular Orbital (MO) Theory, when atoms combine to form molecules, the atomic orbitals of each constituent atom combine and interact with the atomic orbitals of other atoms in the molecule to form molecular orbitals, so the electronic configuration of the molecule is different than the sum of the electronic configurations of each constituent atom. Thankfully, the laws of quantum mechanics constrain the number of probable and stable electronic configurations a given molecule can have, and computational approaches, both classical and quantum, have been devised to help solve for the possible electronic configurations for molecules of various sizes.

An electronic configuration of special interest is the lowest energy, or "ground state" electron configuration. In this challenge, you and your team will create a hybrid algorithm with both classical and quantum elements that can take information about a molecule's energy in the form of a Hamiltonian and use it to find the ground state energy level of the molecule.

## Challenge Overview

For molecules, the electronic structure of a given molecule can be completely described by specifying its Hamiltonian $H$, a matrix (more generally, an operator) that completely specifies how the energy of the molecule evolves over time.

Chemists represent the molecular Hamiltonian in terms of quantum field theory (QFT) operators that operate directly on the electrons (a type of fermion), and a Hamiltonian of this form, known as a fermionic Hamiltonian, cannot be directly used as input for a quantum computer since it directly operates on electrons instead of the qubits. Thus, it is necessary to map the QFT operators to quantum gates, converting the fermionic Hamiltonian into a qubit (or spin) Hamiltonian that can operate on qubits on a quantum computer. The first part of the challenge will involve implementing the Jordan-Wigner Transform to map an arbitrary fermionic Hamiltonian to a qubit Hamiltonian.

Once we have the spin Hamiltonian for a molecule, the ground state energy of the molecule is given by the smallest eigenvector of the spin Hamiltonian matrix. To find this eigenvalue, we will construct a high-level implementation of the Variational Quantum Eigensolver, a hybrid quantum and classical optimization algorithm that, in particular, can find eigenvalues of a given matrix.

Finally, the last part of the challenge involves optimizing your JWT and VQE algorithms to use fewer qubits and run more effectively.

## Second Quantization

For the sake of completeness, the following section is included to provide an introduction to the structure of the fermionic Hamiltonian and does not need to be understood fully. To conveniently specify states with multiple particles, we will use the occupation number representation. Let $\{|1\rangle, |2\rangle,...\}$ be a set of single-particle states that form an orthonormal basis for the single-particle Hilbert space $\mathcal{H}^{(1)}$, and let $n_j$ particles be in state $|j\rangle$. Then, we can represent a multi-particle state by

$$|n_1, n_2,... \rangle$$

where the numbers $n_j$ are referred to as occupation numbers. Since fermions obey the Pauli-exclusion principle, for the rest of this challenge, we can assume each $n_j \in \{0, 1\}$. Additionally, we will assume that the set

$$S = \{|n_1, n_2,... \rangle : j \in \mathbb{Z}_{\geq 1}, n_j \in \{0, 1\}\}$$

forms a complete orthogonal basis for the vector space consisting of the fermion multi-particle quantum states (i.e. multi-particle states that are antisymmetric under exchange). More specifically, $S$ forms a complete orthogonal basis for the fermionic Fock space

$$\mathcal{F}_A = \mathcal{H}^{(0)} \oplus \mathcal{H}^{(1)} \oplus \mathcal{H}^{(2)}_A \oplus \mathcal{H}^{(3)}_A \oplus...$$

where $\oplus$ is a direct sum and $\mathcal{H}^{(j)}_A$ is the antisymmetric subspace of the $j$-particle Hilbert space. Thus, any physical state of fermions $| \Psi \rangle$ can be written in the form

$$|\Psi \rangle = \sum_{(n_1, n_2, ...) \in \{0 ,1\}^\infty} \alpha_{n_1, n_2,...}|n_1, n_2,... \rangle$$

We now use the occupation number representation to introduce a formalism for constructing and describing fermionic Hamiltonians, known as the second quantization formalism. A fermionic Hamiltonian is described by a system of $N$ fermionic modes (in our case, this corresponds to a system with $N$ fermions). Each fermionic mode is given by the set of annihilation operators $\{\hat{a}_1, ..., \hat{a}_{N-1}\}$. The adjoint $\hat{a}_j^\dagger$ of an annihilation operator $\hat{a}_j$ is known as a creation operator, and the creation and annihilation operators are collectively referred to as fermionic ladder operators. While fully understanding creation and annihilation operators would require quantum field theory, we can try to understand them by considering how they would act on a multi-particle quantum state. On an arbitrary multi-particle quantum state, the action of the fermionic creation and annihilation operators is given by

$$\hat{a}_j |n_1, n_2,... n_{j - 1}, 1, n_{j + 1}... \rangle = \gamma \delta_{0, n_j}|n_1, n_2,... n_{j - 1}, 0, n_{j + 1}... \rangle$$

$$\hat{a}_j^\dagger |n_1, n_2,... n_{j - 1}, 0, n_{j + 1}... \rangle = \gamma \delta_{1, n_j}|n_1, n_2,... n_{j - 1}, 1, n_{j + 1}... \rangle$$

where $\delta_{\mu, \nu}$ is the Kronecker delta function and $\gamma = e^{i\pi \sum_{k < j} n_k}$ is an unimportant phase factor. Thus, on the vacuum state $|\varnothing \rangle = |00...0 \rangle$ corresponding to no fermions present,

$$\hat{a}_0^\dagger |\varnothing \rangle = |10...0 \rangle$$

$$\hat{a}_0 |10...0 \rangle = |\varnothing \rangle$$

so the fermionic creation operator creates a fermion in state $|j\rangle$, and the annihilation operator removes a fermion in a state $|j\rangle$. In the second quantization, the fermionic Hamiltonian $H$ is

$$H = E_{core} + \sum_{p, q} h_{pq} \hat{a}_p^\dagger \hat{a}_q + \frac{1}{2} \sum_{p, q, r, s} g_{pqrs} \hat{a}_p^\dagger \hat{a}_q^\dagger \hat{a}_r \hat{a}_s$$

where $p, q, r, s \in \{1,..., M\}$ are the spin-orbital modes, $M$ is the total number of single-particle spin-orbitals (basis modes), $E_{core}$ is the core energy (usually from nuclear-nuclear repulsion) of the molecule, $h$ is a matrix containing the energies for nucleus-electron interactions, and $g$ is a 4-dimentional array (a rank 4 tensor) containing the electron–electron Coulomb interaction energies.

## Part 1: The Jordan-Wigner Transform

From the previous section, we know that the fermionic Hamiltonian for a given molecule $H$ can be written in the form

$$H = E_{core} + \sum_{p, q} h_{pq} \hat{a}_p^\dagger \hat{a}_q + \frac{1}{2} \sum_{p, q, r, s} g_{pqrs} \hat{a}_p^\dagger \hat{a}_q^\dagger \hat{a}_r \hat{a}_s$$

where $p, q, r, s \in \{1,..., M\}$ are the spin-orbital modes, $M$ is the total number of single-particle spin-orbitals, $E_{core}$ is the core energy of the molecule, $h$ is a matrix containing the energies for nucleus-electron interactions, $g$ is a rank 4 tensor containing the electron–electron Coulomb interaction energies, and the $\hat{a}_j^\dagger$ and $\hat{a}_j$ are the creation and annihilation operators acting on the $j$th orbital of the molecule. To simulate fermions on a quantum computer, we must first find a good representation of the creation and annihilation operators on the Hilbert space of qubits so that we can map the fermionic Hamiltonian to a qubit Hamiltonian. Qubit Hamiltonians are written in terms of the identity matrix $I$ and the Pauli matrices $X$, $Y$, and $Z$, so in other words we must find a substitution $\varphi: \{\hat{a}_1, ..., \hat{a}_{N-1}, \hat{a}_1^\dagger, ..., \hat{a}_{N-1}^\dagger\} \rightarrow \{I, X, Y, Z\}$ that substitutes $\hat{a}_j^\dagger$ and $\hat{a}_j$ with quantum gates constructed from Pauli matrices.

In this challenge, you and your team will implement the Jordan-Wigner Transform (JWT). Under JWT, an annihilation operator acting on the $j$th fermionic mode is mapped to a Pauli string (a product of $N$-qubit Pauli operators) as follows:

$$\hat{a}_j \mapsto \frac{1}{2}(X_j - iY_j)Z_1 \cdot Z_2 \cdot\cdot\cdot Z_{j-1}$$

Recall that the $X_j$, $Y_j$, and $Z_j$ are the Pauli matrices acting on the $j$th qubit, so for $A_j \in \{X_j, Y_j, Z_j\}$, we have

$$A_j = I \otimes \cdot\cdot\cdot \otimes I \otimes A \otimes I \otimes \cdot\cdot\cdot \otimes I$$

where $A \in \{X, Y, Z\}$ is the single-qubit operator corresponding to $A_j$ acting at location $j$.

Similarly, it can be shown that for the creation operator acting on the $j$th fermionic mode,

$$\hat{a}_j^\dagger \mapsto \frac{1}{2}(X_j + iY_j)Z_1 \cdot Z_2 \cdot\cdot\cdot Z_{j-1}$$

### Challenge 1

Using the equations in this section, rewrite the fermionic Hamiltonian in terms of qubit operators $I$, $X$, $Y$, and $Z$ to transform the fermionic Hamiltonian to a qubit Hamiltonian. Note that Pauli operators generally do not commute. Then, program a function `convert_hamiltonian` that can convert a given fermionic Hamiltonian to a qubit Hamiltonian using the Jordan-Wigner Transform. `convert_hamiltonian` should have three inputs `ecore`, `h`, and `g`, which correspond to $E_{core}$, $h$, and $g$ respectively from the fermionic Hamiltonian. `convert_hamiltonian` should have a single output, a qubit Hamiltonian in the form of a `SparsePauliOp` array.  Do not import any other libraries other than those already imported without permission.

Write your expression for the qubit Hamiltonian here:
$$H = $$

In [1]:
# In case Qiskit was not already installed (along with the AER simulator)
# !pip install qiskit
# !pip install qiskit-ibm-runtime
# !pip install qiskit_aer

In [76]:
# General imports
import matplotlib.pyplot as plt
import numpy as np
from qiskit.quantum_info import SparsePauliOp

def convert_hamiltonian(ecore: float, h: np.ndarray, g: np.ndarray) -> SparsePauliOp:
    # Insert your code here.

SyntaxError: incomplete input (353976874.py, line 6)

## Part 2: Variational Quantum Eigensolver

Next, to find the ground state of the electron configuration of the molecule, you have to find the ground state of the qubit Hamiltonian for that molecule using the Variational Quantum Eigensolver (VQE). VQE has four components, some of which have a classical workflow and others that have a quantum workflow.
- An operator: For our purposes, this is the qubit Hamiltonian $H$ for the molecule we are trying to find the ground state energy of. Generally, the operator could be any property of the system you want to optimize. Notice that optimizing this operator is the same as trying to find the smallest eigenvalue of the operator, or the operator's ground state.
- An "ansatz": Ansatz means "approach" in German, and the VQE ansatz is a quantum circuit that prepares a quantum state approximating the eigenvalue you are seeking. If you know additional information about the Hamiltonian that you are trying to optimize, you can use that when constructing your ansatz in order to converge more quickly to the ground state.
- An estimator: This is a way to approximate the expectation value of the operator $H$ over the current variational quantum state. For our purposes, the estimator can be thought of as the "cost function" we are trying to minimize.
- A classical optimizer: A classical optimization algorithm, like gradient descent, that can vary parameters in order to minimize the cost function.

### Challenge 2:

Implement a fully functioning VQE algorithm named `vqe` that can be run on the Qiskit AER quantum simulator for various Hamiltonians. `vqe` should have only one input, the molecular Hamiltonian `H` in the form of a `SparsePauliOp` array. `vqe` should have only one output, the smallest eigenvalue of the Hamiltonian. Additionally, your VQE implementation should plot the quantum circuit for your Ansatz.

Your ansatz should be constructed in the `build_ansatz` algorithm. `build_ansatz` should output a `QuantumCircuit` that represents your best guess for the ground state energy before minimization. Consider using the `qiskit.circuit.library` to find a good ansatz.

The cost function for your VQE algorithm should be encapsulated in the `cost_func` quantum algorithm. It should output an estimate for the lowest eigenvalue of the input Hamiltonian.

Do not use any additional libraries without permission.

In [ ]:
# General imports
import scipy
import matplotlib as mpl
import numpy as np

# SciPy minimizer routine
from scipy.optimize import minimize

# Plotting functions
import matplotlib.pyplot as plt

# Runtime imports
import qiskit
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, Session
# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In [ ]:
def vqe(H: SparsePauliOp) -> float:
    # Initialize the simulator and estimator down below

    # Insert your code that uses cost_func and build_ansatz, minimizes your cost function, and returns the value obtained through minimization
    # of the cost function, which should ideally be very close to the smallest eigenvalue of the input Hamiltonian

In [ ]:
# You may want this import to "optimize" your ansatz- better ansatz = better score
# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

def build_ansatz(H):
    # Insert your code to construct the ansatz here

In [ ]:
def cost_func(params, ansatz, H, estimator, cost_history_dict):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        H (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    # Insert your cost function code here

## Bonus

The naive implementation of JWT created in this challenge can convert even very complicated fermionic Hamiltonians into spin Hamiltonians. However, the naive JWT implementation outputs a spin Hamiltonian with way too many Pauli strings that have nonzero values in the `SparsePauliOp` output of the algorithm for the outputted spin Hamiltonian to then be analyzed in meaningful ways. The biggest contributor to Pauli string length and number is the electron-electron interaction coefficient tensor $g$, and in practice, it becomes necessary to decompose the $g$ tensor in the JWT algorithm before doing the full calculation and expansion of the molecular Hamiltonian terms that involve the electron-electron interaction energy coefficients.

### Challenge 3:

Optimize your implementation of JWT so that it generates the "simplest" (it should have the smallest number of nonzero Pauli string elements in the final result) spin Hamiltonian. Consider using the Cholesky decomposition. This is an open-ended challenge, so the simpler your final spin Hamiltonian, the better you will perform on this challenge.

In [ ]:
# You may go back and modify your code above to optimize it and/or implement optimization code, such as a Cholesky decomposition function, down here